### Reranking Hybrid Search Statergies

Re-ranking is a second-stage filtering process in retrieval systems, especially in RAG pipelines, where we:

1. First use a fast retriever (like BM25, FAISS, hybrid) to fetch top-k documents quickly.

2. Then use a more accurate but slower model (like a cross-encoder or LLM) to re-score and reorder those documents by relevance to the query.

👉 It ensures that the most relevant documents appear at the top, improving the final answer from the LLM.

In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser 
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

/Users/sauravmajumdar/Developer/AI/RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## load text file
loader=TextLoader("langchain_sample.txt")
raw_docs=loader.load()

# Split text into document chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(raw_docs)
docs


[Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.'),
 Document(metadata={'source': 'langchain_sample.txt'}, page_content='Retrieval-Augmented Generation (RAG) is a powerful technique where external knowledge is retrieved and passed into the prompt to ground LLM responses. LangChain makes it easy to implement RAG using vector databases like FAISS, Chroma, and Pinecone.\nBM25 is a traditional 

In [3]:
## user query
query="How can i use langchain to build an application with memory and tools?"

In [4]:
### FAISS and Huggingface model Embeddings

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(docs,embedding_model)
retriever=vectorstore.as_retriever(search_kwargs={"k":8})

In [5]:
## OpenAI Embedding
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings()
vectorstore_openai=FAISS.from_documents(docs,embeddings)
retriever_openai=vectorstore_openai.as_retriever(search_kwargs={"k":8})

In [6]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x148e6f530>, search_kwargs={'k': 8})

In [7]:
retriever_openai

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x16c6feb70>, search_kwargs={'k': 8})

In [8]:
## prompt and use the llm
from langchain.chat_models import init_chat_model
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
llm=init_chat_model("openai:gpt-4o-mini")
llm

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x16c74ce60>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x16c74d490>, root_client=<openai.OpenAI object at 0x16c626d50>, root_async_client=<openai.AsyncOpenAI object at 0x16c74cec0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [9]:
# Prompt Template
prompt = PromptTemplate.from_template("""
You are a helpful assistant. Your task is to rank the following documents from most to least relevant to the user's question.

User Question: "{question}"

Documents:
{documents}

Instructions:
- Think about the relevance of each document to the user's question.
- Return a list of document indices in ranked order, starting from the most relevant.

Output format: comma-separated document indices (e.g., 2,1,3,0,...)
""")

In [ ]:
retrieved_docs=retriever_openai.invoke(query)
retrieved_docs

[Document(id='0fbe9cfd-b902-4786-b53c-637c37b36b03', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='6609ec0c-ba1b-442c-b693-fa5ad8bfc8e2', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='063e8afa-eb40-4162-8cdc-44931e41ee40', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging F

In [12]:
chain=prompt| llm | StrOutputParser()
chain

PromptTemplate(input_variables=['documents', 'question'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. Your task is to rank the following documents from most to least relevant to the user\'s question.\n\nUser Question: "{question}"\n\nDocuments:\n{documents}\n\nInstructions:\n- Think about the relevance of each document to the user\'s question.\n- Return a list of document indices in ranked order, starting from the most relevant.\n\nOutput format: comma-separated document indices (e.g., 2,1,3,0,...)\n')
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions

In [15]:
doc_lines = [f"{i+1}. {doc.page_content}" for i, doc in enumerate(retrieved_docs)]
formatted_docs = "\n".join(doc_lines)

In [16]:
doc_lines

['1. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.',
 '2. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.',
 '3. LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.',
 '4. Dense retrieval uses embeddings to match query and documents in a vector space. This allows capturing semantic meaning, making it usefu

In [17]:
formatted_docs

'1. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.\n2. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.\n3. LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.\n4. Dense retrieval uses embeddings to match query and documents in a vector space. This allows capturing semantic meaning, making it useful for fuzz

In [18]:
response=chain.invoke({"question":query,"documents":formatted_docs})
response

'2,1,3,5,6,4'

In [19]:
# Step 5: Parse and rerank
indices = [int(x.strip()) - 1 for x in response.split(",") if x.strip().isdigit()]
indices

[1, 0, 2, 4, 5, 3]

In [20]:
retrieved_docs

[Document(id='0fbe9cfd-b902-4786-b53c-637c37b36b03', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='6609ec0c-ba1b-442c-b693-fa5ad8bfc8e2', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='063e8afa-eb40-4162-8cdc-44931e41ee40', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging F

**What does this reranking code do?**

The LLM returned a ranked list of document indices (like `[2, 1, 3, 0, ...]`). This code:

1. **Takes those indices** and uses them to reorder `retrieved_docs`
2. **Filters out invalid indices** - if an index is out of bounds (negative or >= length), it's skipped
3. **Creates a new list** with documents in the LLM's ranked order

**Example:**
- `retrieved_docs` = [doc0, doc1, doc2, doc3] (original order)
- `indices` = [2, 0, 1] (LLM says: doc2 is most relevant, then doc0, then doc1)
- `reranked_docs` = [doc2, doc0, doc1] (reordered by relevance)

This ensures the most relevant documents appear first for the final answer!


In [23]:
# Reorder documents based on LLM's ranking
reranked_docs = [retrieved_docs[i] for i in indices if 0 <= i < len(retrieved_docs)]

# Simpler way to do it
# reranked_docs = []
# for idx in indices:
#     # Only add valid indices (within bounds)
#     if 0 <= idx < len(retrieved_docs):
#         reranked_docs.append(retrieved_docs[idx])

reranked_docs

[Document(id='6609ec0c-ba1b-442c-b693-fa5ad8bfc8e2', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='0fbe9cfd-b902-4786-b53c-637c37b36b03', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='063e8afa-eb40-4162-8cdc-44931e41ee40', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging F

In [ ]:
# Step 6: Show results
print("\n📊 Final Reranked Results:")
for i, doc in enumerate(reranked_docs, 1):
    print(f"\nRank {i}:\n{doc.page_content}")


📊 Final Reranked Results:

Rank 1:
LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.

Rank 2:
LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.
Memory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.

Rank 3:
LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.

Rank 4:
FAISS is a popular library used for fast approximate nearest neighbor search in high-dimensional sp

## FlashRank Reranker

**FlashRank** is an ultra-lite & super-fast Python library for re-ranking search results using state-of-the-art cross-encoders. It's much faster than using an LLM for reranking and provides better accuracy than simple vector similarity.

**How it works:**
1. Use a fast retriever (like FAISS) to get top-k documents
2. FlashRank re-ranks them using a cross-encoder model
3. Returns the most relevant documents at the top

Notes:
**Available compressors you can use:**
- `FlashrankRerank` — Cross-encoder reranking (what you're using)
- `LLMChainExtractor` — Uses an LLM to extract relevant parts
- `LLMChainFilter` — Uses an LLM to filter documents
- `EmbeddingsFilter` — Filters by embedding similarity threshold
- `CohereRerank` — Cohere's reranking API
- `LLMLinguaCompressor` — Compresses documents using LLMLingua
- `DocumentCompressorPipeline` — Chain multiple compressors together

In [ ]:
# Import FlashRank components
from langchain_community.document_compressors import FlashrankRerank
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever


In [29]:
# Create FlashRank reranker compressor
# FlashRank uses cross-encoder models to score query-document pairs
flashrank_compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")

# Wrap the base retriever with compression retriever
# This will automatically rerank results using FlashRank
compression_retriever = ContextualCompressionRetriever(
    base_compressor=flashrank_compressor,
    base_retriever=retriever_openai  # Using the OpenAI retriever we set up earlier
)


INFO:flashrank.Ranker:Downloading ms-marco-MiniLM-L-12-v2...
ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:03<00:00, 7.00MiB/s]


In [34]:
# Retrieve and rerank documents using FlashRank
flashrank_docs = compression_retriever.invoke(query)
print("\n👨‍💻 Question:", query)
print("=" * 100)
print(f"\n📊 FlashRank Reranked Results ({len(flashrank_docs)} documents):\n")
print("=" * 100)
for i, doc in enumerate(flashrank_docs, 1):
    print(f"\nRank {i}:\n{doc.page_content}\n")
    print("-" * 100)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"



👨‍💻 Question: How can i use langchain to build an application with memory and tools?

📊 FlashRank Reranked Results (3 documents):


Rank 1:
LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.
Memory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.

----------------------------------------------------------------------------------------------------

Rank 2:
LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.

----------------------------------------------------------------------------------------------------

Rank 3:
LangChain integrates with many third-party services such 

### Compare: Original Retrieval vs FlashRank Reranking

Let's see the difference between the original retrieval order and FlashRank's reranked order:


In [41]:
# Get original retrieval results (before reranking)
original_docs = retriever_openai.invoke(query)

print("🔹 Original Retrieval Order (Top 3):")
for i, doc in enumerate(original_docs[:3], 1):
    print(f"\n{i}. {doc.page_content[:150]}...")

print("\n\n🔹 FlashRank Reranked Order (Top 3):")
for i, doc in enumerate(flashrank_docs[:3], 1):
    print(f"\n{i}. {doc.page_content[:150]}...")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


🔹 Original Retrieval Order (Top 3):

1. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accu...

2. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to ...

3. LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different mod...


🔹 FlashRank Reranked Order (Top 3):

1. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accu...

2. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to ...

3. LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment 

### Use FlashRank Reranker in a RAG Chain

Now let's use the FlashRank reranked retriever in a complete RAG pipeline:


In [39]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain

# Create prompt for RAG
rag_prompt = PromptTemplate.from_template("""
Answer the question based on the context below. If the context is not relevant, say "I don't know"

Context:
{context}

Question: {input}
""")

# Create document chain
document_chain = create_stuff_documents_chain(llm=llm, prompt=rag_prompt)

# Create full RAG chain with FlashRank reranked retriever
rag_chain = create_retrieval_chain(
    retriever=compression_retriever,  # Using FlashRank reranked retriever
    combine_docs_chain=document_chain
)


In [ ]:
# Query the RAG chain with FlashRank reranking
result = rag_chain.invoke({"input": query})
print("Question:", query)
print("\nAnswer:")
print(result["answer"])


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Question: How can i use langchain to build an application with memory and tools?

Answer:
I don't know.
